### BANK TRANSACTION DATA PIPELINE & FRAUD DETECTION WITH MongoDB 

1. Design the MongoDB Schema
    a. Define the collection and their respective fields.
        - Accounts and credit limits as well as credit balance etc.
        - Banks & Countries and minimum balance for customers and the charges for certain transfers.
        - The exchange rates.
        - Merchants
        - Transactions
2. Populate the accounts, banks and merchants collection according to the validators. Implement minimum balances with different curre
3. Simulate transaction data with logic below.
    a.  For each transaction, check if the sender and receiver are valid accounts.
    b.  If valid, update the sender and receiver balance accordingly and insert into collection.
    c.  Specify the transaction type be it a deposit, atm withdrawal, purchase(online or in-store), cheques(withdrawal or deposit), charge, POS, payment or transfer.
    d. The account should not be below the balance for balance account and not go over for credit accounts.
    e. If not valid, log the error and skip the transaction.
    f. Every transaction should be unique.
    g. Implement the logic to calculate the exchange rate for each transaction based on the exchange rates collection.
    h. Implement the logic to calculate the fees for each transaction based on the banks and countries collection.
4. Data Transformation
5. Data Loading into MongoDB
6. Fraud detection logic/cases.
7. Data Analytics
8. Automation.


In [2]:
from pymongo import MongoClient
from pymongo.errors import CollectionInvalid

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Define the database
db = client["banking_system"]

# Define the Accounts collection schema with validation
accounts_schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["account_id", "account_holder_name", "account_type", "current_balance", "minimum_balance", "bank_id", "currency"],
        "properties": {
            "account_id": {"bsonType": "int"},
            "account_holder_name": {"bsonType": "string"},
            "account_type": {"bsonType": "string", "enum": ["savings", "checking", "credit"]},
            "credit_limit": {"bsonType": "double"},
            "credit_balance": {"bsonType": "double", "minimum": 0.0},
            "current_balance": {"bsonType": "double", "minimum": 0.0},
            "minimum_balance": {"bsonType": "double", "minimum": 0.0},
            "bank_id": {"bsonType": "int"},
            "currency": {"bsonType": "string"},
        }
    }
}

# Define the Banks collection schema with validation
banks_schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["bank_id", "bank_name", "country", "transfer_fee", "minimum_balance"],
        "properties": {
            "bank_id": {"bsonType": "int"},
            "bank_name": {"bsonType": "string"},
            "country": {"bsonType": "string"},
            "transfer_fee": {"bsonType": "double", "minimum": 0.0},
            "minimum_balance": {"bsonType": "double", "minimum": 0.0},
        }
    }
}


# Define the Exchange Rates collection schema with validation
exchange_rates_schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["currency_pair", "exchange_rate", "timestamp"],
        "properties": {
            "currency_pair": {"bsonType": "string"},
            "exchange_rate": {"bsonType": "double", "minimum": 0.0},
            "timestamp": {"bsonType": "date"},
        }
    }
}

# Define the Merchants collection schema with validation
merchants_schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["merchant_id", "merchant_name", "merchant_type", "currency"],
        "properties": {
            "merchant_id": {"bsonType": "int"},
            "merchant_name": {"bsonType": "string"},
            "merchant_type": {"bsonType": "string", "enum": ["online", "retail"]},
            "currency": {"bsonType": "string"},
        }
    }
}

# Define the Transactions collection schema with validation
transactions_schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["transaction_id", "sender_account_id", "receiver_account_id", "amount", "transaction_type", "timestamp", "status"],
        "properties": {
            "transaction_id": {"bsonType": "int"},
            "sender_account_id": {"bsonType": "int"},
            "receiver_account_id": {"bsonType": "int"},
            "amount": {"bsonType": "double", "minimum": 0.0},
            "transaction_type": {
                "bsonType": "string",
                "enum": ["deposit", "withdrawal", "purchase", "cheque", "charge", "POS", "payment", "transfer"],
            },
            "timestamp": {"bsonType": "date"},
            "status": {"bsonType": "string", "enum": ["success", "failed"]},
            "fee": {"bsonType": "double", "minimum": 0.0},
        }
    }
}

# Create the collections with validation
try:
    db.create_collection("accounts", validator=accounts_schema)
    db.create_collection("banks", validator=banks_schema)
    db.create_collection("countries", validator=countries_schema)
    db.create_collection("exchange_rates", validator=exchange_rates_schema)
    db.create_collection("merchants", validator=merchants_schema)
    db.create_collection("transactions", validator=transactions_schema)

    print("Collections created with schema validation!")
except CollectionInvalid:
    print("Collections already exist. Validation rules have been updated.")


Collections created with schema validation!


In [4]:
from pymongo import MongoClient
from datetime import datetime

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Define the database
db = client["banking_system"]

# Example documents to insert
accounts_doc = {
    "account_id": 1,
    "account_holder_name": "John Doe",
    "account_type": "savings",
    "credit_limit": 5000.00,
    "credit_balance": 0.00,
    "current_balance": 1500.00,
    "minimum_balance": 100.00,
    "bank_id": 101,
    "currency": "USD"
}

banks_doc = {
    "bank_id": 101,
    "bank_name": "First National Bank",
    "country": "USA",
    "transfer_fee": 2.50,
    "minimum_balance": 100.00
}

countries_doc = {
    "country_code": "US",
    "country_name": "United States",
    "currency": "USD",
    "exchange_rates": {
        "EUR": 0.85,
        "GBP": 0.75
    }
}

exchange_rates_doc = {
    "currency_pair": "USD/EUR",
    "exchange_rate": 0.85,
    "timestamp": datetime(2024, 10, 12, 12, 0, 0)  # Convert to datetime
}

merchants_doc = {
    "merchant_id": 201,
    "merchant_name": "Online Store",
    "merchant_type": "online",
    "currency": "USD"
}

transactions_doc = {
    "transaction_id": 301,
    "sender_account_id": 1,
    "receiver_account_id": 2,
    "amount": 100.00,
    "transaction_type": "transfer",
    "timestamp": datetime(2024, 10, 12, 12, 30, 0),  # Convert to datetime
    "status": "success",
    "fee": 2.00
}

# Insert documents into their respective collections
db.accounts.insert_one(accounts_doc)
db.banks.insert_one(banks_doc)
db.countries.insert_one(countries_doc)
db.exchange_rates.insert_one(exchange_rates_doc)
db.merchants.insert_one(merchants_doc)
db.transactions.insert_one(transactions_doc)

print("Documents inserted successfully!")



Documents inserted successfully!


In [ ]:
accounts_schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["account_id", "account_holder_name", "account_type", "current_balance", "minimum_balance", "bank_id", "currency"],
        "properties": {
            "account_id": {"bsonType": "int"},
            "account_holder_name": {"bsonType": "string"},
            "account_type": {"bsonType": "string", "enum": ["savings", "checking", "credit"]},
            "credit_limit": {"bsonType": "double"},
            "credit_balance": {"bsonType": "double", "minimum": 0.0},
            "current_balance": {"bsonType": "double", "minimum": 0.0},
            "minimum_balance": {"bsonType": "double", "minimum": 0.0},
            "bank_id": {"bsonType": "int"},
            "currency": {"bsonType": "string"},
        }
    }
}